# JuliaのTips集
このページはJuliaでの実装におけるTips (詰まったところの解決策)をまとめたものである．体系的にまとまってはいない．

## 1. 関数名の!記号
単なる**慣習**として関数への入力を変更する場合に!を付ける．

関数内で配列を変更する場合には注意が必要である．以下に入力された配列を同じサイズの要素1の配列で置き換える，ということを目的として書かれた2つの関数がある．違いは`v`の後に`[:]`としているかどうかである．

In [1]:
function wrong!(a::Array)
    a = ones(size(a))
end

function right!(a::Array)
    a[:] = ones(size(a))
end

right! (generic function with 1 method)

実行すると`wrong!`の場合には入力された配列が変更されていないことがわかる．

In [3]:
using Random
v = rand(2, 2)
println("v : ", v)

wrong!(v)
println("wrong : ", v)

right!(v)
println("right : ", v)

v : [0.41750278591651857 0.03675917069399426; 0.1360642202272484 0.6578991307730526]
wrong : [0.41750278591651857 0.03675917069399426; 0.1360642202272484 0.6578991307730526]
right : [1.0 1.0; 1.0 1.0]


## 2. 配列の1次元化
配列を一次元化(flatten)する方法．まずは3次元配列を作成する．

In [7]:
B = rand(2, 2, 2)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.997107  0.0827965
 0.394606  0.847174

[:, :, 2] =
 0.0305417  0.448929
 0.270553   0.601419

用意されている`flatten`を素直に用いると次のようになる．

In [2]:
import Base.Iterators: flatten
collect(flatten(B))

8-element Array{Float64,1}:
 0.25323335359382226
 0.8475284309218816
 0.46170218524594486
 0.3908851101558619
 0.2049406129688014
 0.20068478557126213
 0.29072826846396627
 0.5193322837086369

ただし，単に`B[:]`とするだけでもよい．

In [3]:
B[:]

8-element Array{Float64,1}:
 0.25323335359382226
 0.8475284309218816
 0.46170218524594486
 0.3908851101558619
 0.2049406129688014
 0.20068478557126213
 0.29072826846396627
 0.5193322837086369

## 3. 行列の行・列ごとの正規化
シミュレーションにおいてニューロン間の重み行列を行あるいは列ごとに正規化 (weight normalization)する場合がある．これは各ニューロンへの入力の大きさを同じにする働きや重みの発散を防ぐ役割がある．以下では行ごとの和を1にする．

In [5]:
W = rand(3,3)

3×3 Array{Float64,2}:
 0.947321  0.526878  0.464399
 0.290191  0.875268  0.279707
 0.319338  0.976441  0.375153

In [6]:
Wnormed = W ./ sum(W, dims=1)

3×3 Array{Float64,2}:
 0.608486  0.221509  0.414916
 0.186396  0.367978  0.249904
 0.205118  0.410513  0.33518

In [7]:
println(sum(Wnormed, dims=1))

[1.0 1.0 1.0]


## 4. 行列の結合 (concatenate)
行列の結合はMATLABに近い形式で行うことができる．まず，2つの行列A, Bを用意する．

In [1]:
A = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [2]:
B = [4 5 6; 7 8 9]

2×3 Array{Int64,2}:
 4  5  6
 7  8  9

### 4.1 水平結合 (Horizontal concatenation)
`hcat`を使うやり方と，`[ ]`を使うやり方がある．

In [13]:
H1 = hcat(A,B)

2×5 Array{Int64,2}:
 1  2  4  5  6
 3  4  7  8  9

In [11]:
H2 = [A B]

2×5 Array{Int64,2}:
 1  2  4  5  6
 3  4  7  8  9

なお，MATLABのように次のようにすると正しく結合はされない．

In [16]:
H3 = [A, B]

2-element Array{Array{Int64,2},1}:
 [1 2; 3 4]
 [4 5 6; 7 8 9]

### 4.2 垂直結合 (Vertical concatenation)

In [6]:
V1 = vcat(A, B')

5×2 Array{Int64,2}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [19]:
V2 = [A; B']

5×2 Array{Int64,2}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [21]:
[V2 [A;B']]

5×4 Array{Int64,2}:
 1  2  1  2
 3  4  3  4
 4  7  4  7
 5  8  5  8
 6  9  6  9

## 5. 配列に新しい軸を追加
要はnumpyでの`A[None, :]`や`A[np.newaxis, :]`のようなことがしたい場合．やや面倒だが，`reshape`を使うか，`[CartesianIndex()]`を用いる．

In [14]:
v = rand(3)

3-element Array{Float64,1}:
 0.3487808667518093
 0.717776723811032
 0.3369916406749496

In [24]:
newaxis = [CartesianIndex()]
v1 = v[newaxis, :]

1×3 Array{Float64,2}:
 0.348781  0.717777  0.336992

## 6. Array{Array{Float64, x},1}をArray{Float64, x+1}に変換
numpyでは`array([matrix for i in range()])`などを用いると，1次元配列のリストを2次元配列に変換できた．Juliaでも同様にする場合は`hcat(...)`や`cat(...)`を用いる．

In [40]:
A1 = [i*rand(3) for i=1:5]

println("Type : ", typeof(A1))
println("Size : ", size(A1))

Type : Array{Array{Float64,1},1}
Size : (5,)


In [41]:
A2 = hcat(A1...)'

println("Type : ", typeof(A2))
println("Size : ", size(A2))

Type : LinearAlgebra.Adjoint{Float64,Array{Float64,2}}
Size : (5, 3)


以下は多次元配列の場合．`cat(...)`で配列を結合し，`permitedims`で転置する．

In [54]:
B1 = [i*rand(3, 4, 5) for i=1:6]

println("Type : ", typeof(B1))
println("Size : ", size(B1))

Type : Array{Array{Float64,3},1}
Size : (6,)


In [59]:
B2 = permutedims(cat(B1..., dims=4), [4, 1, 2, 3])

println("Type : ", typeof(B2))
println("Size : ", size(B2))

Type : Array{Float64,4}
Size : (6, 3, 4, 5)


## 7. 二項分布 (bernoulli distribution)のサンプリング

In [1]:
p = 0.7
N = 100

100

In [2]:
using BenchmarkTools

In [14]:
@benchmark floor.(p .+ rand(N))

BenchmarkTools.Trial: 
  memory estimate:  1.91 KiB
  allocs estimate:  7
  --------------
  minimum time:     676.823 ns (0.00% GC)
  median time:      692.073 ns (0.00% GC)
  mean time:        727.852 ns (2.96% GC)
  maximum time:     4.498 μs (81.11% GC)
  --------------
  samples:          10000
  evals/sample:     164

In [10]:
println(sum(floor.(p .+ rand(N)) .== 1.0) / N) 

0.7


In [13]:
@benchmark 1.0f0 * (p .≥ rand(N))

BenchmarkTools.Trial: 
  memory estimate:  1.55 KiB
  allocs estimate:  6
  --------------
  minimum time:     853.704 ns (0.00% GC)
  median time:      885.185 ns (0.00% GC)
  mean time:        942.825 ns (2.37% GC)
  maximum time:     14.357 μs (92.66% GC)
  --------------
  samples:          10000
  evals/sample:     54

In [12]:
println(sum(1.0f0 * (p .≥ rand(N)) .== 1.0) / N) 

0.75


## 8. Roth's column lemma

Roth's column lemmaは，例えば，$A, B, C$が与えられていて，$X$を未知とするときの方程式 $AXB = C$を考えると，この方程式は

$$
(B^\top \otimes A)\text{vec}(X) = \text{vec}(AXB)=\text{vec}(C)
$$

の形に書き下すことができる，というものである．$\text{vec}(\cdot)$はvec作用素（行列を列ベクトル化する作用素）である．`vec(X) = vcat(X...)`で実現できる．Roth's column lemmaを用いれば，$AXB = C$の解は

$$
X = \text{vec}^{-1}\left((B^\top \otimes A)^{-1}\text{vec}(C)\right)
$$

として得られる．ただし，$\text{vec}(\cdot)^{-1}$は列ベクトルを行列に戻す作用素(inverse of the vectorization operator)である．`reshape()`で実現できる．2つの作用素をまとめると，

$$
\begin{align}
\text{vec} &: R^{m\times n}\to R^{mn}\\
\text{vec}^{−1} &: R^{mn}\to R^{m×n}
\end{align}
$$

であり，$\text{vec}^{−1}\left(\text{vec}(X)\right)=X\ (\text{for all}\ X\in R^{m\times n})，\text{vec}\left(\text{vec}^{−1}(x)\right)=x\ (\text{for all}\ x \in R^{mn})$となる．

In [1]:
using LinearAlgebra, Kronecker, Random

In [2]:
m = 4
A = randn(m, m)
B = randn(m, m)
C = convert(Array{Float64}, reshape(1:16, (m, m)))

4×4 Matrix{Float64}:
 1.0  5.0   9.0  13.0
 2.0  6.0  10.0  14.0
 3.0  7.0  11.0  15.0
 4.0  8.0  12.0  16.0

In [3]:
vec(X) = vcat(X...)

vec (generic function with 1 method)

In [4]:
X = reshape((B' ⊗ A)^-1 * vec(C), (m, m))

4×4 Matrix{Float64}:
   9.50815    64.8954   104.476  -94.925
 -12.2845    -83.4891  -134.881  124.289
 -15.2998   -103.743   -167.919  155.905
 -28.1594   -192.07    -309.381  281.71

In [5]:
A * X * B

4×4 Matrix{Float64}:
 1.0  5.0   9.0  13.0
 2.0  6.0  10.0  14.0
 3.0  7.0  11.0  15.0
 4.0  8.0  12.0  16.0

### 9. "splat" operator ... 
https://docs.julialang.org/en/v1/base/base/#...

### 10. broadcastingの回避

In [1]:
foo(a,b) = sum(a) + b

foo (generic function with 1 method)

In [6]:
println(foo.(Ref([1,2]),[3,4,5]))
println(foo.(([1,2],), [3,4,5]))
println(foo.([[1,2]], [3,4,5]))

[6, 7, 8]
[6, 7, 8]
[6, 7, 8]


### 11. `var`の使用

In [1]:
var"log(θ+μ)" = 10

10